## 准备数据

In [35]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0

    return (x, y), (x_test, y_test)

In [36]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


## 建立模型

In [37]:
class myModel:
    def __init__(self):
        ####################
        '''声明模型对应的参数'''
        self.W1 = tf.Variable(tf.random.normal(shape=(28 * 28, 128), stddev=0.01), trainable=True)
        self.b1 = tf.Variable(tf.zeros(shape=(128,)), trainable=True)
        self.W2 = tf.Variable(tf.random.normal(shape=(128, 10), stddev=0.01), trainable=True)
        self.b2 = tf.Variable(tf.zeros(shape=(10,)), trainable=True)
        ####################
    def __call__(self, x):
        ####################
        '''实现模型函数体，返回未归一化的logits'''
        ####################
        x_flattened = tf.reshape(x, [-1, 784])
        h1 = tf.matmul(x_flattened, self.W1) + self.b1
        activated_h1 = tf.tanh(h1)
        logits = tf.matmul(activated_h1, self.W2) + self.b2
        return logits

model = myModel()

optimizer = optimizers.Adam()

## 计算 loss

In [38]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.01*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [40]:
train_data, test_data = mnist_dataset()
for epoch in range(100):
    loss, accuracy = train_one_step(model, optimizer,
                                    tf.constant(train_data[0], dtype=tf.float32),
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model,
                      tf.constant(test_data[0], dtype=tf.float32),
                      tf.constant(test_data[1], dtype=tf.int64))



print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 2.2703397 ; accuracy 0.58781666
epoch 1 : loss 2.2699556 ; accuracy 0.58883333
epoch 2 : loss 2.2695692 ; accuracy 0.58986664
epoch 3 : loss 2.2691803 ; accuracy 0.59061664
epoch 4 : loss 2.2687893 ; accuracy 0.5916333
epoch 5 : loss 2.2683957 ; accuracy 0.59255
epoch 6 : loss 2.2679994 ; accuracy 0.59315
epoch 7 : loss 2.267601 ; accuracy 0.5937667
epoch 8 : loss 2.2672 ; accuracy 0.5945
epoch 9 : loss 2.2667966 ; accuracy 0.59576666
epoch 10 : loss 2.2663906 ; accuracy 0.5965833
epoch 11 : loss 2.265982 ; accuracy 0.59723336
epoch 12 : loss 2.2655709 ; accuracy 0.59795
epoch 13 : loss 2.265157 ; accuracy 0.59855
epoch 14 : loss 2.2647407 ; accuracy 0.5993
epoch 15 : loss 2.2643213 ; accuracy 0.59995
epoch 16 : loss 2.2638996 ; accuracy 0.60048336
epoch 17 : loss 2.263475 ; accuracy 0.6013333
epoch 18 : loss 2.2630475 ; accuracy 0.60181665
epoch 19 : loss 2.262617 ; accuracy 0.6023333
epoch 20 : loss 2.2621844 ; accuracy 0.6027333
epoch 21 : loss 2.2617486 ; accuracy 0.